In [51]:
import numpy as np

In [52]:
def PickWinner(teams, prob=.5):
    """
    Function that randomly selects the winner of a game.
    
    teams: list (2,) of two teams playing each other
    prob: float of how likely it is historically that the first team listed 
    in teams wins the game.
    """

    if np.random.rand() < prob: #If the random number between 0 and 1 is less than the 
        return teams[0]         #specified probability, then return the first team listed
    else:
        return teams[1]
    
def Bracket():
    bracket = [["Auburn", "ALST/SFPA"], ["Louisville", "Creighton"], ["Michigan", "UCSD"], ["Texas A&M", "Yale"], \
              ["Ole Miss", "SDSU/UNC"], ["Iowa State", "Lipscomb."], ["Marquette", "New Mexico"], ["Michigan St.", "Bryant"], \
              ["Florida", "Norfolk St."], ["UConn", "Oklahoma"], ["Memphis", "Colorado St."], ["Maryland", "Grand Canyon"], \
              ["Missouri", "Drake"], ["Texas Tech", "UNC Wilmington"], ["Kansas", "Arkansas"], ["St. John's", "Omaha"], \
              ["Duke", "AMER/MSM"], ["Miss St.", "Baylor"], ["Oregon", "Liberty"], ["Arizona", "Akron"], \
              ["BYU", "VCU"], ["Wisconsin", "Montana"], ["Saint Mary's", "Vanderbilt"], ["Alabama", "Robert Morris"], \
              ["Houston", "SIUE"], ["Gonzaga", "Georgia"], ["Clemson", "McNeese"], ["Purdue", "High Point"], \
              ["Illinois", "TEX/XAV"], ["Kentucky", "Troy"], ["UCLA", "Utah St."], ["Tennessee", "Wofford"]]
    return bracket

def NextRound(bracket, filename, num_round):
    
    if num_round == 2:
        with open(filename, 'a') as myfile:
            myfile.write("SWEET SIXTEEN: \n")
    elif num_round == 3:
        with open(filename, 'a') as myfile:
            myfile.write("ELITE EIGHT: \n")
    elif num_round == 4:
        with open(filename, 'a') as myfile:
            myfile.write("FINAL FOUR: \n")
    elif num_round == 5:
        with open(filename, 'a') as myfile:
            myfile.write("NATIONAL CHAMPIONSHIP: \n")
            
    if len(bracket) == 1:
        with open(filename, 'a') as myfile:
            myfile.write("NATIONAL CHAMPION: \n")
            myfile.write(bracket[0][0])
        return 
    else:
        next_bracket = []
        for i in range(0, len(bracket), 2):
            two = [PickWinner(bracket[i]), PickWinner(bracket[i+1])]
            next_bracket.append(two)
            
            with open(filename, 'a') as myfile:
                for team in two:
                    myfile.write(team)
                    myfile.write("\n")
                    
        return NextRound(next_bracket, filename, num_round + 1)

    
def FirstRound(initial_bracket):
    second_round = []
    real_second_round = []
    for i in range(len(initial_bracket)):
        second_round.append(PickWinner(initial_bracket[i], prob=seed(i)))
    
    for i in range(0,len(second_round),2):
        real_second_round.append([second_round[i], second_round[i+1]])
    return real_second_round
        
        
def seed(i):
    if i in [0,8,16,24]:
        return .992
    elif i in [1,9,17,25]:
        return .5
    elif i in [2,10,18,26]:
        return .65
    elif i in [3,11,19,27]:
        return .645
    elif i in [4,12,20,28]:
        return .625
    elif i in [5,13,21,29]:
        return .85
    elif i in [6,14,22,30]:
        return .6
    elif i in [7,15,23,31]:
        return .938

In [53]:
def read_file(filename):
    second = []
    sweet = []
    elite = []
    four = []
    final = []
    champ = []

    with open(filename, 'r') as myfile:
        line = myfile.readline()
        while line != "SWEET SIXTEEN:":
            line = myfile.readline().strip()
            second.append(line)
        second.pop()
        
        while line != "ELITE EIGHT:":
            line = myfile.readline().strip()
            sweet.append(line)
        sweet.pop()
        
        while line != "FINAL FOUR:":
            line = myfile.readline().strip()
            elite.append(line)
        elite.pop()
        
        while line != "NATIONAL CHAMPIONSHIP:":
            line = myfile.readline().strip()
            four.append(line)
        four.pop()
        
        while line != "NATIONAL CHAMPION:":
            line = myfile.readline().strip()
            final.append(line)
        final.pop()
        champ.append(myfile.readline().strip())
    
    totals = [second, sweet, elite, four, final, champ]
    return totals

def accuracy(predicted, actual):
    round_accuracy = []
    rounds = len(actual)
    total_acc = 0
    total_length = 0
    game_acc = 0
    for i in range(rounds):
        acc = 0
        total_length += len(actual[i])
        
        if len(actual[i]) == 1:
            if predicted[i][0] == actual[i][0]:
                acc += 1
                total_acc += 1
        else:
            for j in range(0,len(actual[i]),2):
                if predicted[i][j] in actual[i] and predicted[i][j+1] in actual[i]:
                    game_acc += 1
                    acc += 2
                    total_acc += 2
                elif predicted[i][j] in actual[i] or predicted[i][j+1] in actual[i]:
                    acc += 1
                    total_acc += 1
                else:
                    acc += 0
                    total_acc += 0
        round_accuracy.append(acc/len(predicted[i]))
        
    return round_accuracy, total_acc/total_length, game_acc/(total_length//2)




In [54]:
def generate(num_brackets: int):
    for i in range(num_brackets):
        initial_bracket = Bracket()
        testrun = FirstRound(initial_bracket)
        filename = f"brackets/round{str(i)}.txt"
        with open(filename, 'w') as myfile:
            myfile.write("SECOND ROUND: \n")
            for game in testrun:
                for i in range(2):
                    myfile.write(game[i])
                    myfile.write("\n")
        NextRound(testrun, filename, 2)


In [ ]:
# generate(5000) ##DO NOT TOUCH, THIS WILL CREATE MORE BRACKETS, THEY ARE ALREADY GENERATED

In [55]:
def score_all(actual: list, num_brackets: int):
    accuracies = []
    for i in range(num_brackets):
        long_total = read_file("brackets/round"+str(i)+".txt")
        accuracies.append(accuracy(long_total, actual))
    
    rounds = [tup[0] for tup in accuracies]
    percents = [tup[1:] for tup in accuracies]
    perc_arr = np.array(percents).reshape(num_brackets,2)
    return rounds, perc_arr

def score_one(actual: list, filename: str):
    predicted = read_file(filename)
    accuracies = accuracy(predicted, actual)
    return accuracies

In [8]:
#The list "mine" represents the actual outcomes every round
#This will have to be updated every round and run again to get complete
#scores per round of how the random brackets are doing.
#r_score, p_scores = score_all(mine)

In [ ]:
so_far = [["Auburn", "Creighton", "Michigan", "Texas A&M", "Ole Miss", "Iowa State", \
           "New Mexico", "Michigan St.", "Florida", "UConn", "Colorado St.", "Maryland", \
          "Drake", "Texas Tech", "Arkansas", "St. John's", "Duke", "Baylor", "Oregon", "Arizona", \
          "BYU", "Wisconsin", "Saint Mary's", "Alabama", "Houston", "Gonzaga", "McNeese", "Purdue", "Illinois", \
          "Kentucky", "UCLA", "Tennessee"], ["Auburn", "Michigan", "Ole Miss", "Michigan St.", "Florida", \
                                             "Maryland", "Texas Tech", "Arkansas", "Duke", "Arizona", "BYU", "Alabama", \
                                                "Houston", "Purdue", "Kentucky", "Tennessee"], ["Auburn", "Michigan St.", \
                                       "Florida", "Texas Tech", "Duke", "Alabama", "Houston", "Tennessee"], \
                        ["Auburn", "Florida", "Duke", "Houston"]]

r_sc, perc_a = score_all(so_far, 5000)

In [57]:
#finding the best bracket out of all 2000 by overall percentage of games
#picked correctly.
#Print the index of the best bracket

def find_best_bracket(num_brackets: int, r_sc, percent):
    """Finds the best overall score, and the bracket with that score.
    Returns the best overall score and the bracket details like the index as well
    as the best score and the percentage of game matchups that the bracket predicted
    correctly. Much more needs to be done here I think."""
    best_overall_score = np.max(np.array(percent).reshape(num_brackets,2)[:,0]) # best round percentage no matter what, regardless of bracket
    bracket_with_best_overall_score = np.argmax(np.array(percent).reshape(num_brackets,2)[:,0]) # bracket with the best 
    return best_overall_score, percent[bracket_with_best_overall_score], bracket_with_best_overall_score

find_best_bracket(5000, r_sc, perc_a)

(np.float64(0.7678571428571429),
 array([0.76785714, 0.60714286]),
 np.int64(2411))

In [58]:
round_acc = score_one(so_far, "brackets/round2411.txt")
print(round_acc)

([0.84375, 0.75, 0.5], 0.7678571428571429, 0.6071428571428571)


In [43]:
print(r_sc[3007])

[0.9375, 0.6875]
